In [4]:
import cv2
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import re
# Open the video file
model = keras.models.load_model('vgg16_mdl1.hdf5')
test_data_dir = "C:/Users/lecle/Documents/Kitti_project/archive/testing/"

for video in os.listdir(test_data_dir):
    cap = cv2.VideoCapture(test_data_dir + video)

    ret, frame = cap.read()
    # print(frame.shape)
    output_size = frame.shape
    print(output_size)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    print(video)
    out_name = video.replace(".mp4", "")

    out = cv2.VideoWriter(f'./video_test/{out_name}_test.mp4', fourcc, 20.0, (output_size[1], output_size[0]), )

    # Define a sliding window size
    window_size = 128
    # Process each frame of the video
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()

        if ret:
            # Convert the frame to grayscale and resize it
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_resized = cv2.resize(frame_gray, (128, 128))

            # Pad the frame to ensure that the sliding window covers the entire frame
            frame_padded = np.pad(frame_resized, ((
                0, window_size), (0, window_size)), 'constant', constant_values=0)

            # Initialize the output mask for the frame
            mask = np.zeros_like(frame_resized)

            # Slide a window over the frame and apply the model on each window
            for i in range(0, frame_padded.shape[0] - window_size, window_size):
                for j in range(0, frame_padded.shape[1] - window_size, window_size):
                    # Extract the window
                    window = frame_padded[i:i+window_size, j:j+window_size]
                    window = cv2.cvtColor(window, cv2.COLOR_GRAY2BGR)
                    window_mask = model.predict(
                        window.reshape(1, window_size, window_size, 3))
                    window_mask = (window_mask > 0.5).astype(
                        np.uint8).reshape(window_size, window_size)

                    mask[i:i+window_size, j:j+window_size] += window_mask
                    
                    zero_image = np.zeros_like(mask)
                    mask = np.dstack((mask, zero_image, zero_image))
                    mask = np.asarray(mask, np.float32)
                    mask *= 255

            img = np.asarray(frame, np.float32)
            mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR) 

            mask = tf.image.resize(mask, [output_size[0], output_size[1]])
            new_img = cv2.addWeighted(img, 0.8, mask.numpy(), 0.2, 0)

            new_img = tf.keras.preprocessing.image.array_to_img(new_img)
            new_img = np.asarray(new_img)

            out.write(new_img)

        else:
            break

    # Release the resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()


(720, 1280, 3)
challenge.mp4
1/1 [==============================] - 0s 28ms/step
(720, 1280, 3)
challenge_video.mp4
1/1 [==============================] - 0s 20ms/step
(720, 1280, 3)
harder_challenge_video.mp4
1/1 [==============================] - 0s 20ms/step
(720, 1280, 3)
project_video.mp4
1/1 [==============================] - 0s 35ms/step
(540, 960, 3)
solidWhiteRight.mp4
1/1 [==============================] - 0s 43ms/step
(540, 960, 3)
solidYellowLeft.mp4
1/1 [==============================] - 0s 36ms/step
